In [11]:
!pip install peft==0.11.1 transformers==4.36.2 accelerate==0.25.0 --no-cache-dir
!pip install diffusers==0.24.0 --no-cache-dir


In [12]:
import torch
from diffusers import StableDiffusionPipeline
from peft import LoraConfig, get_peft_model
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using:", device)


Using: cuda


In [13]:
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to(device)

prompt = "A cute cat sitting on Mars wearing a space helmet"


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  8.66it/s]


In [14]:
before_img = pipe(prompt, num_inference_steps=25).images[0]

os.makedirs("outputs", exist_ok=True)
before_path = "outputs/before_lora.png"
before_img.save(before_path)

print("Saved before image:", before_path)


  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:02<00:00, 10.01it/s]


Saved before image: outputs/before_lora.png


In [15]:
# Find valid LoRA injection points inside SD 1.5 UNet
target_modules = []

for name, module in pipe.unet.named_modules():
    if any(x in name for x in ["to_q", "to_k", "to_v", "to_out"]):
        target_modules.append(name)

print("Discovered LoRA-compatible modules:")
for m in target_modules:
    print("  -", m)


Discovered LoRA-compatible modules:
  - down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q
  - down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_k
  - down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v
  - down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out
  - down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0
  - down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.1
  - down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_q
  - down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_k
  - down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_v
  - down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out
  - down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0
  - down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.1
  - down_blocks.0.attentions.1.transformer_blocks.0.attn1.to_q
  - down_blocks.0.attentions.1.transformer_blocks.0.attn1.to_k
  - down_blocks.0.attentions.1.transformer_blocks.0.attn1.to_v

In [18]:
from diffusers import StableDiffusionPipeline
import torch, os

device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to(device)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  6.39it/s]


In [20]:
import torch
from diffusers import StableDiffusionPipeline

lora = torch.load("adapters/tinyllama_magicbrush_lora/unet_lora.pth")

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

unet = pipe.unet

# Inject LoRA into UNet
for name, module in unet.named_modules():
    if name in lora:
        A, B = lora[name]
        module.weight.data += (B @ A) * 0.1   # scale factor

print("LoRA loaded!")

prompt = "a sunset over a lake"
output = pipe(prompt, num_inference_steps=25).images[0]
output.save("after_lora.png")


/tmp/ipykernel_21819/3635222447.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora = torch.load("adapters/tinyllama_magicbrush_lora/unet_lora.pth")


FileNotFoundError: [Errno 2] No such file or directory: 'adapters/tinyllama_magicbrush_lora/unet_lora.pth'